In [2]:
!uv run python -m pytest "test.py::test_fsdp_with_itself" -q

...F                                                                     [100%]
=================================== FAILURES ===================================
___________________ test_fsdp_with_itself[bfloat16-bfloat16] ___________________

param_dtype = 'bfloat16', reduce_dtype = 'bfloat16'

    @pytest.mark.parametrize("param_dtype", [None, "bfloat16"])
    @pytest.mark.parametrize("reduce_dtype", ["float32", "bfloat16"])
    def test_fsdp_with_itself(
        param_dtype: Literal["bfloat16", "float32"] | None,
        reduce_dtype: Literal["bfloat16", "float32"] | None,
    ):
>       run_distributed_test(
            _test_fsdp_with_itself,
            param_dtype=param_dtype,
            reduce_dtype=reduce_dtype,
            world_size=2,
        )

test.py:116: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
test.py:82: in run_distributed_test
    mp.start_processes(  # type: ignore[attr-defined,no-untyped-call]
.venv/lib/python3.12/site-packa

**как можно заметить fsdp2 сам против себя не выдает стабильные результаты** так что поменял тесты чтобы они принимали погрешность до 1e-3

In [3]:
!uv run python -m pytest "test.py::test_fsdp" -q

....                                                                     [100%]
=============================== warnings summary ===============================
.venv/lib/python3.12/site-packages/torch/jit/_script.py:362: 14 warnings
  /workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/jit/_script.py:362: DeprecationWarning: `torch.jit.script_method` is deprecated. Please switch to `torch.compile` or `torch.export`.
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
4 passed, 14 warnings in 42.10s


In [6]:
!uv run torchrun --standalone --nproc-per-node=2 train.py --fsdp fsdp2 --num-steps-to-profile 3 --snapshots-dir snapshots/fsdp2 --traces-dir traces/fsdp2

W0226 21:14:14.450000 69311 torch/distributed/run.py:852] 
W0226 21:14:14.450000 69311 torch/distributed/run.py:852] *****************************************
W0226 21:14:14.450000 69311 torch/distributed/run.py:852] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0226 21:14:14.450000 69311 torch/distributed/run.py:852] *****************************************
2026-02-26 21:14:19,947 - __main__ - INFO - model size: 6163712 parameters.
2026-02-26 21:14:19,977 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-26 21:14:19,980 - root - INFO - Preparing c4_test dataset from ./c4_test
/workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/profiler/profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.To keep e

In [7]:
!uv run torchrun --standalone --nproc-per-node=2 train.py --fsdp effdl --num-steps-to-profile 3 --snapshots-dir snapshots/effdl --traces-dir traces/effdl

W0226 21:14:29.651000 69463 torch/distributed/run.py:852] 
W0226 21:14:29.651000 69463 torch/distributed/run.py:852] *****************************************
W0226 21:14:29.651000 69463 torch/distributed/run.py:852] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0226 21:14:29.651000 69463 torch/distributed/run.py:852] *****************************************
2026-02-26 21:14:35,337 - __main__ - INFO - model size: 6163712 parameters.
2026-02-26 21:14:35,367 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-26 21:14:35,369 - root - INFO - Preparing c4_test dataset from ./c4_test
/workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/profiler/profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.To keep e

<img src="images/image.png" width="900">

<img src="images/image2.png" width="900">

как видно мемори снепшоты получаются очень похожими